# 系列変換モデルで学習する

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

try:
    from google.colab import files
    print('Google Colab. 上での実行です')
    print('「ファイルを選択」から、notebook/dataset フォルダの中身をすべてを選択し、アップロードしてください')
    print('===========')
    files.upload()
    !mkdir dataset
    !mv * ./dataset
    print('次に notebook/common フォルダの中身をすべてを選択し、アップロードしてください')
    print('===========')
    files.upload()
    !mkdir common
    !mv *.py ./common
except:
    print('ローカル環境での実行です')


from dataset import sequence
from common.optimizer import Adam
from common.trainer import Trainer
from common.util import eval_seq2seq
from common.seq2seq import Seq2seq # seq2seq
from common.attention_seq2seq import AttentionSeq2seq # アテンション付きseq2seq
from common.attention_biseq2seq import AttentionBiSeq2seq # エンコーダ側LSTMが双方向になったアテンション付きseq2seq

In [ ]:
# データの読み込み
(x_train, t_train), (x_test, t_test) = sequence.load_data('date.txt')
char_to_id, id_to_char = sequence.get_vocab()

# ハイパーパラメータの設定
vocab_size = len(char_to_id)
wordvec_size = 16
hidden_size = 256
batch_size = 256
max_epoch = 20 # 計算時間削減のため20epochとしているが，十分な学習を行うには更に増やす必要がある

### モデルの選択
モデルを切り替えて、結果を比較してみましょう

In [ ]:
# model = Seq2seq(vocab_size, wordvec_size, hidden_size)
# model = AttentionSeq2seq(vocab_size, wordvec_size, hidden_size)
model = AttentionBiSeq2seq(vocab_size, wordvec_size, hidden_size)

### 学習

In [ ]:
# 最適化手法の設定
optimizer = Adam()

# 学習のオブジェクトを生成
trainer = Trainer(model, optimizer)

# 学習のループ
acc_list = []
loss_list = []
for epoch in range(max_epoch):
    
    # trainデータで1epoch分の計算
    trainer.fit(x_train, t_train, max_epoch=1,batch_size=batch_size)

    # testデータで精度を確認する
    correct_num = 0
    for i in range(len(x_test)):
        question, correct = x_test[[i]], t_test[[i]]
        verbose = i < 10
        correct_num += eval_seq2seq(model, question, correct, id_to_char, verbose) 

    # 精度算出
    acc = float(correct_num) / len(x_test)
    acc_list.append(acc)
    print('val acc %.3f%%' % (acc * 100))

    # loss算出
    loss = model.forward(x_test, t_test)
    loss_list.append(loss)
    print('val loss %.3f' % (loss))
    
    # 重み保存
    model.save_params()



In [ ]:
# Accuracyの描画
x = np.arange(len(acc_list))
plt.plot(x, acc_list, marker='o')
plt.xlabel('epochs')
plt.ylabel('accuracy')
plt.ylim(-0.05, 1.05)
plt.show()

# Lossの描画
plt.plot(x, loss_list, marker='o')
plt.xlabel('epochs')
plt.ylabel('loss')
# plt.ylim(-0.05, 0.1)
plt.show()

### [演習]
* Seq2seq、AttentionSeq2seq、AttentionBiSeq2seqのそれぞれの場合を計算し、結果を比較してみましょう。